# Linear Regression

### Pytorch Realisation of Linear Regression Algorithm

In [1]:
import torch
from torch.nn import Linear, functional as F
from torch.optim import SGD

In [2]:
class LinearRegression(Linear):
    def __init__(self, in_features, out_features = 1, bias = True):
        super().__init__(in_features= in_features, out_features=out_features, bias=bias)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.00005)
    
    def fit(self, train_dl, num_epochs=100):
        for epoch in range(num_epochs+1):
            for x_batch, y_batch in train_dl:
                self.prediction = self(x_batch.float())
                loss = F.mse_loss(self.prediction, y_batch.float())
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')
        

<br>

#### Test on Boston Houses Dataset

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('data.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,0.98765,0.0,12.50,0,0.561,6.980,89.0,2.0980,3,320,23.0,396.00,12.00,12.0
507,0.23456,0.0,12.50,0,0.561,6.980,76.0,2.6540,3,320,23.0,343.00,25.00,32.0
508,0.44433,0.0,12.50,0,0.561,6.123,98.0,2.9870,3,320,23.0,343.00,21.00,54.0
509,0.77763,0.0,12.70,0,0.561,6.222,34.0,2.5430,3,329,23.0,343.00,76.00,67.0


In [5]:
if df.isna().sum().sum() > 0:
    df.dropna(inplace=True)

In [6]:
df.isna().sum().sum()

0

In [7]:
X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

In [8]:
X_ = torch.from_numpy(X)
y_ = torch.from_numpy(y)

In [9]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

tensor_dataset = TensorDataset(X_, y_)
data_loader = DataLoader(tensor_dataset, batch_size=5, shuffle = True)

In [10]:
l = LinearRegression(13, 1)

In [11]:
l.fit(data_loader)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  # Remove the CWD from sys.path while we load stuff.
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\torch\autograd\__init__.py:156: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
c:\users\user\appdata\local\programs

Epoch 0, Loss: 14676.40625
Epoch 10, Loss: 7627.32861328125
Epoch 20, Loss: 6046.310546875
Epoch 30, Loss: 2830.946044921875
Epoch 40, Loss: 1727.21337890625
Epoch 50, Loss: 59.443145751953125
Epoch 60, Loss: 93.16427612304688
Epoch 70, Loss: 9.024128913879395
Epoch 80, Loss: 104.44635772705078
Epoch 90, Loss: 124.3644790649414
Epoch 100, Loss: 49.80086135864258


<br><br>

### Numpy Realisation of Linear Regression Algorithm

In [12]:
class LinearRegression_numpy():
    def __init__(self, in_features, out_features, bias = True):
        self.weight = np.random.rand(in_features, out_features)
        self.bias = np.random.rand(out_features)
    
    def __loss_fn(self, y_predicted, target):
        return np.square(y_predicted - target)
    
    def predict(self, X):
        return np.matmul(X, self.weight) + self.bias
    
    def fit(self, X, y, num_epochs, lr):
        mean_loss = 0
        for epoch in range(num_epochs):
            for index in range(len(X)):
                x_value, y_value = X[index], y[index]
                loss = self.__loss_fn(self.predict(x_value), y_value) 
                mean_loss += loss
                
                direction = 2 * (self.predict(x_value) - y_value) 
                self.weight -= lr * (direction * x_value.transpose()).reshape(self.weight.shape)
                self.bias -= lr * direction
            
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {mean_loss/(10 * len(X))}')
                mean_loss = 0
    
    def score(self, X, y):
        mean_loss = 0
        for index in range(len(X)):
            x_value, y_value = X[index], y[index]
            loss = self.__loss_fn(self.predict(x_value), y_value) 
            mean_loss += np.sqrt(loss)
        return mean_loss / len(X)

In [13]:
w = LinearRegression_numpy(13, 1)

In [14]:
w.fit(X, y, 100, 5e-7)

Epoch 0, Loss: [129.2189948]
Epoch 10, Loss: [129.88233118]
Epoch 20, Loss: [95.35900108]
Epoch 30, Loss: [80.96791143]
Epoch 40, Loss: [71.35603385]
Epoch 50, Loss: [64.4009571]
Epoch 60, Loss: [59.30202126]
Epoch 70, Loss: [55.55420191]
Epoch 80, Loss: [52.79404142]
Epoch 90, Loss: [50.75472819]


In [15]:
w.score(X,y)

array([21.24965349])

In [16]:
w.weight

array([[ 0.09839532],
       [ 0.06714408],
       [ 0.7016684 ],
       [ 0.81581469],
       [ 0.60812591],
       [ 0.30219948],
       [-0.02392468],
       [ 0.24983972],
       [ 0.83417849],
       [ 0.00603077],
       [ 0.08742688],
       [ 0.05724049],
       [-0.09455959]])